In [ ]:
url = "https://api-web.nhle.com/v1/partner-game/CA/now"

r = requests.get(url)

json_data = r.json()
       
# Normalize the JSON data into a DataFrame
df = pd.json_normalize(data = json_data,#data, 
                       record_path = 'games',
                       sep = "_",
                       errors = 'ignore')
df

In [ ]:
url = "https://api-web.nhle.com/v1/partner-game/CA/now"

r = requests.get(url)

json_data = r.json()
       
# Normalize the JSON data into a DataFrame
df = pd.json_normalize(data = json_data,#data, 
                       record_path = 'games',
                       sep = "_")

def explode_and_melt(df, team_type):
    if team_type.upper() == 'HOME':
        team_abbrev_col = 'homeTeam_abbrev'
        team_odds_col = 'homeTeam_odds'
    elif team_type.upper() == 'AWAY':
        team_abbrev_col = 'awayTeam_abbrev'
        team_odds_col = 'awayTeam_odds'
    else:
        raise ValueError("Invalid team type. Use 'HOME' or 'AWAY'.")

    # Explode the respective team's odds column
    df_team_odds = df[['gameId', team_abbrev_col, team_odds_col]].explode(team_odds_col)
    
    # Melt the DataFrame
    df_team_odds = df_team_odds.melt(id_vars=['gameId', team_abbrev_col], var_name='team_type_odds_description', value_name='odds')
    
    # Concatenate the exploded DataFrame with normalized odds values
    df_team_odds = pd.concat([df_team_odds[['gameId', team_abbrev_col, 'team_type_odds_description']], json_normalize(df_team_odds['odds'])], axis=1)
    
    # Renaming columns using rename()
    df_team_odds = df_team_odds.rename(columns={'description': 'bet_description'})

    
    return df_team_odds

# Example usage:
# Assuming df is your original DataFrame

# Explode and process for the home team
home_team_odds = explode_and_melt(df, 'HOME')
home_team_odds = home_team_odds.rename(columns={'homeTeam_abbrev': 'team_abbrev_home',
                                                'value': 'value_home',
                                                'qualifier': 'qualifier_home'})

# Assuming 'column_to_remove' is the name of the column you want to remove
home_team_odds.drop(columns=['team_type_odds_description'], inplace=True)

# Explode and process for the away team
away_team_odds = explode_and_melt(df, 'AWAY')
away_team_odds = away_team_odds.rename(columns={'awayTeam_abbrev': 'team_abbrev_away',
                                                'value': 'value_away',
                                                'qualifier': 'qualifier_away'})

# Assuming 'column_to_remove' is the name of the column you want to remove
away_team_odds.drop(columns=['team_type_odds_description'], inplace=True)

# Merge the DataFrames on 'gameId' and 'bet_description'
merged_df = pd.merge(home_team_odds, away_team_odds, on=['gameId', 'bet_description'], how='outer')

# Convert values in 'qualifier_home' and 'qualifier_away' columns to uppercase
merged_df['qualifier_home'] = merged_df['qualifier_home'].str.upper()
merged_df['qualifier_away'] = merged_df['qualifier_away'].str.upper()

print(merged_df)


# Updated Code
This is the code to use below

In [191]:
import requests
import numpy as np
import pandas as pd
import oracledb
import os
import re
from datetime import datetime, timedelta
from pandas import json_normalize

# Function to convert camel case to snake case
def camel_to_snake(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

# Function to process URLs and return a list of DataFrames
def process_urls(urls):
    all_dataframes = []

    for url in urls:
        r = requests.get(url)
        json_data = r.json()

        betting_partner_data = json_data.get('bettingPartner', {})
        current_odds_date = json_data.get('currentOddsDate', {})
        last_update_utc = json_data.get('lastUpdatedUTC', {})

        df_current_odds_date = pd.DataFrame({'current_odds_date': [current_odds_date]})
        df_last_update_utc = pd.DataFrame({'last_update_utc': [last_update_utc]})

        df_betting_partner = pd.DataFrame(betting_partner_data, index=[0])

        combined_df = pd.concat([df_current_odds_date, df_last_update_utc, df_betting_partner], axis=1)
        combined_df = combined_df[['current_odds_date', 'last_update_utc', 'partnerId', 'country', 'name', 'siteUrl']]
        
        combined_df.rename(columns={'name': 'partner_name'}, inplace = True)
        combined_df['last_update_utc'] = pd.to_datetime(combined_df['last_update_utc'], format='%Y-%m-%dT%H:%M:%SZ')

        all_dataframes.append(combined_df)

    return all_dataframes

# List of URLs
url_list = [
    "https://api-web.nhle.com/v1/partner-game/CA/now",
    "https://api-web.nhle.com/v1/partner-game/US/now",
]

# Process URLs and get a list of DataFrames
betting_partner_df = process_urls(url_list)

# Access individual DataFrames from the list
for index, df in enumerate(betting_partner_df):
    print(f"DataFrame {index + 1}:")
    print(df)
    print("-------------")


DataFrame 1:
  current_odds_date     last_update_utc  partnerId country partner_name  \
0        2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   

                  siteUrl  
0  https://sportradar.com  
-------------
DataFrame 2:
  current_odds_date     last_update_utc  partnerId country partner_name  \
0        2023-12-29 2023-12-30 01:30:40          9     USA   DraftKings   

                                             siteUrl  
0  https://dksb.sng.link/As9kz/720m?_dl=https%3A%...  
-------------


In [192]:
def process_urls(url_list):
    all_dataframes = []

    for url in url_list:
        r = requests.get(url)
        json_data = r.json()

        # Normalize the JSON data into a DataFrame
        df = pd.json_normalize(data=json_data, record_path='games', sep='_')

        def explode_and_melt(df, team_type):
            if team_type.upper() == 'HOME':
                team_abbrev_col = 'homeTeam_abbrev'
                team_odds_col = 'homeTeam_odds'
            elif team_type.upper() == 'AWAY':
                team_abbrev_col = 'awayTeam_abbrev'
                team_odds_col = 'awayTeam_odds'
            else:
                raise ValueError("Invalid team type. Use 'HOME' or 'AWAY'.")

            # Explode the respective team's odds column
            df_team_odds = df[['gameId', team_abbrev_col, team_odds_col]].explode(team_odds_col)

            # Melt the DataFrame
            df_team_odds = df_team_odds.melt(id_vars=['gameId', team_abbrev_col], var_name='team_type_odds_description', value_name='odds')

            # Concatenate the exploded DataFrame with normalized odds values
            df_team_odds = pd.concat([df_team_odds[['gameId', team_abbrev_col, 'team_type_odds_description']], json_normalize(df_team_odds['odds'])], axis=1)

            # Renaming columns using rename()
            df_team_odds = df_team_odds.rename(columns={'description': 'bet_description'})


            return df_team_odds

        # Explode and process for the home team
        home_team_odds = explode_and_melt(df, 'HOME')
        home_team_odds = home_team_odds.rename(columns={'homeTeam_abbrev': 'team_abbrev_home',
                                                        'value': 'value_home',
                                                        'qualifier': 'qualifier_home'})
        home_team_odds.drop(columns=['team_type_odds_description'], inplace=True)

        # Explode and process for the away team
        away_team_odds = explode_and_melt(df, 'AWAY')
        away_team_odds = away_team_odds.rename(columns={'awayTeam_abbrev': 'team_abbrev_away',
                                                        'value': 'value_away',
                                                        'qualifier': 'qualifier_away'})
        away_team_odds.drop(columns=['team_type_odds_description'], inplace=True)

        # Merge the DataFrames on 'gameId' and 'bet_description'
        merged_df = pd.merge(home_team_odds, away_team_odds, on=['gameId', 'bet_description'], how='outer')

        # Convert values in 'qualifier_home' and 'qualifier_away' columns to uppercase
        merged_df['qualifier_home'] = merged_df['qualifier_home'].str.upper()
        merged_df['qualifier_away'] = merged_df['qualifier_away'].str.upper()

        all_dataframes.append(merged_df)

    return all_dataframes

# List of URLs
url_list = [
    "https://api-web.nhle.com/v1/partner-game/CA/now",
    "https://api-web.nhle.com/v1/partner-game/US/now",
]

# Process URLs and get a list of DataFrames
odds_df = process_urls(url_list)

# Access individual DataFrames from the list
for index, df in enumerate(odds_df):
    print(f"DataFrame {index + 1}:")
    print(df)
    print("-------------")

DataFrame 1:
        gameId team_abbrev_home   bet_description  value_home qualifier_home  \
0   2023020546              CBJ  MONEY_LINE_2_WAY       173.0                  
1   2023020546              CBJ        OVER_UNDER      -123.0           O6.5   
2   2023020546              CBJ         PUCK_LINE      -135.0            1.5   
3   2023020547              DET        OVER_UNDER      -128.0           O6.5   
4   2023020547              DET  MONEY_LINE_2_WAY      -115.0                  
5   2023020547              DET         PUCK_LINE       208.0           -1.5   
6   2023020548              FLA        OVER_UNDER      -111.0             O6   
7   2023020548              FLA         PUCK_LINE       163.0           -1.5   
8   2023020548              FLA  MONEY_LINE_2_WAY      -144.0                  
9   2023020549              OTT        OVER_UNDER       108.0             O7   
10  2023020549              OTT  MONEY_LINE_2_WAY      -105.0                  
11  2023020549             

In [193]:
combined_dataframes = []

if len(betting_partner_df) == len(odds_df):
    for i in range(len(betting_partner_df)):
        # Expand single-row DataFrames in betting_partner_df to match the length of odds_df
        expanded_betting_partner = pd.concat([betting_partner_df[i]] * len(odds_df[i]), ignore_index=True)
        
        # Combine expanded_betting_partner with odds_df
        combined_df = pd.concat([expanded_betting_partner, odds_df[i]], axis=1)
        
        # Append to the list
        combined_dataframes.append(combined_df)
    
    # Union all the resultant DataFrames
    final_result = pd.concat(combined_dataframes, ignore_index=True)
else:
    print("Lengths of the lists are different!")

print(final_result)

   current_odds_date     last_update_utc  partnerId country partner_name  \
0         2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
1         2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
2         2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
3         2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
4         2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
5         2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
6         2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
7         2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
8         2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
9         2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
10        2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
11        2023-12-29 2023-12-30 01:30:11          8     CAN   Sportradar   
12        20

In [194]:
final_result = final_result.fillna('')

# Convert all columns to string data type
final_result = final_result.astype(str)

# Apply camel_to_snake function to all column names
final_result.columns = final_result.columns.map(camel_to_snake)

In [196]:
import oracledb
import os

# Oracle database connection details
def read_config_file(file_path):
    config = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, *value_parts = line.strip().split('=')
            value = '='.join(value_parts)
            config[key] = value
    return config

# Replace 'path/to/config.txt' with the actual file path
config_file_path = '/Users/jeremybutt/Desktop/credentials.txt'

# Read the configuration from the file
config = read_config_file(config_file_path)

# Set the variables for username, password, and DSN
username = config.get('Oracle_Username')
password = config.get('Oracle_Password')
dsn = config.get('dsn')

# Construct the connection string
connection = oracledb.connect(user=username, password=password, dsn=dsn)

# Create a cursor object
cursor = connection.cursor()

# # Define the target table name
table_name = 'NHL_GAME_ODDS_DAILY_LOAD'

# Drop the table if it already exists
drop_query = f"DROP TABLE {table_name} PURGE"
try:
    cursor.execute(drop_query)
    print(f"Dropped table {table_name}")
except oracledb.DatabaseError as e:
    error_code, error_message = e.args[0].code, e.args[0].message
    if error_code != 942:  # ORA-00942: table or view does not exist
        print(f"Error occurred while dropping table {table_name}: {error_message}")
        
# Read the SQL file
sql_file = r"/Users/jeremybutt/nhl_analysis/02_Source_Code/table_create_scripts/NHL_GAME_ODDS_DAILY_LOAD.sql"
with open(sql_file, 'r') as file:
    create_table_query = file.read()
    
print(create_table_query)

# # Create the table
cursor.execute(create_table_query)
print(f"Created table {table_name}")


# Convert dataframe to a list of tuples
data_tuples = [tuple(row) for row in final_result.values]

# Prepare the INSERT query with placeholders for the columns
insert_query = f"INSERT INTO {table_name} " \
               f"({', '.join(final_result.columns)}) " \
               f"VALUES ({', '.join([':' + str(i+1) for i in range(len(final_result.columns))])})"

# Insert data into the target table
cursor.executemany(insert_query, data_tuples)
print(f"Inserted {len(data_tuples)} rows into {table_name}")

# Commit the changes
connection.commit()

# Read the SQL script
update_file = r"/Users/jeremybutt/nhl_analysis/02_Source_Code/update_scripts/NHL_GAME_ODDS_DAILY_UPDATE.sql"
with open(update_file, 'r') as file:
    script = file.read()

# Split the script into individual statements
statements = script.split(';')

# Execute each statement
for statement in statements:
    if statement.strip():  # Skip empty statements
        try:
            cursor.execute(statement)
            rows_affected = cursor.rowcount
            print(f"Rows affected: {rows_affected}")
        except oracledb.DatabaseError as e:
            error, = e.args
            error_message = error.message
            print(f"Error executing statement: {error_message}")

# Commit the changes
connection.commit()

# Close the cursor and the database connection
cursor.close()
connection.close()

OperationalError: DPY-6005: cannot connect to database.
DPY-6000: Listener refused connection. (Similar to ORA-12506)